# RI and VI

VI(S, U) = H(S|U) + H(U|S)  S - 算法结果  U - 真实结果    H(S|U) - 过分割 split_error   H(U|S) - 欠分割 merger_error

In [5]:
import gala
import numpy as np

import gala.evaluate as ev
from skimage.measure import label, regionprops


# 计算88张测试集的RI and VI

# "./iron_segment/result_total/best_unet_result/" -> unet_result_evaluation_mini
# "./iron_experiments/result_total/best_unet_result_final/" -> unet_result_final_evaluation_mini

# "./iron_experiments/result_total/best_unet_result_focalloss_1/" -> unet_result_focalloss_1_evaluation_mini
# "./3D-UNet/result_total/experiment_unet3d_01/" -> experiment_unet3d_01_evaluation_mini   3d unet
# "./3D-UNet/result_total/experiment_rsunet3d_01/"  -> experiment_rsunet3d_01_evaluation_mini   3d  rs_unet  superhuman
# "./iron_segment_stage3/result_total/best_track_last_unet_dilate_reloss"  ->  best_track_last_unet_dilate_reloss_evaluation_mini

# "./iron_segment_stage3/result_total/best_track_binary_unet_variant6/" -> best_track_binary_unet_variant6_evaluation_mini  压缩路径
# "./iron_segment_stage3/result_total/best_track_binary_unet_variant/" -> best_track_binary_unet_variant1_evaluation_mini  扩张路径
# "./Unet+BDCLSTM/data/test/ffc_result_total/" -> ffc_evaluation_mini  ffc
# "./Unet+BDCLSTM/data/test/unet_bdclstm_result/" -> unet_bdclstm_result_evaluation_mini  ffc


import time, os, cv2

def eval_RI_VI(results_dir, outAd):
    '''
    @param: results_dir: 待评价算法结果图所在地址
    @param：outADd：评价结果存放地
    '''
#     results_dir = "./iron_experiments/result_total/best_unet_result_final/"
    results_path =  sorted(os.listdir(results_dir))
#     print("images count: ", len(results_path))

    gt_dir = "./Unet+BDCLSTM/data/test/manual/"
    RI = 0
    adjust_RI = 0
    merger_error = 0
    split_error = 0
    VI = 0 

    mRI = 0
    mad_RI = 0
    m_merger_error = 0
    m_split_error = 0
    mVI = 0
    count = 0
#     outAd = "./evaluation/mini_RI_VI/unet_result_final_evaluation_mini.txt"
    out = open(outAd, "a")   # 将结果输出至out_lg.txt
    out.write("RI, mRI, adjust_RI, m_adjust_RI, merger_error, m_merger_error, split_error, m_split_error, VI, mVI" + '\n')
    out.close()
    for r in results_path:
        name = r.split(".")[0]
        gt_path = os.path.join(gt_dir, name+".png")
        if os.path.exists(gt_path):
#             print("image", name)
            count += 1
            result = cv2.imread(os.path.join(results_dir, r), 0)
            gt = cv2.imread(gt_path, 0)

            shape = result.shape

            result = result[400:1000, 500:1100]
            gt = gt[400:1000, 500:1100]

            result, num_result = label(result, background=255, neighbors=4, return_num=True)
            gt, num_gt = label(gt, background=255, neighbors=4, return_num=True)


            print(result.shape, " gt ", num_gt, " result ", num_result) 
    #         result = morphology.skeletonize(result / 255) * 255

            start_time = time.time()

            # false merges(缺失), false splits（划痕）
            merger_error, split_error = ev.split_vi(result, gt)
            VI = merger_error + split_error
            RI = ev.rand_index(result, gt)
            adjust_RI = ev.adj_rand_index(result, gt)

            m_merger_error += merger_error
            m_split_error += split_error
            mRI += RI
            mVI += VI
            mad_RI += adjust_RI

            end_time = time.time()

            print(r, " --> RI : ", RI, "adjustRI : ", adjust_RI, " VI : ", VI, " merger_eroor : ", merger_error, " split_error : ", split_error, " costs ", str(end_time-start_time))

            out = open(outAd, "a")   # 将结果输出至out_lg.txt
            # "RI, mRI, adjust_RI, m_adjust_RI, merger_error, m_merger_error, split_error, m_split_error, VI, mVI"
            line = str(RI)+","+str(mRI/count)+","+str(adjust_RI)+","+str(mad_RI/count)+","+str(merger_error)+","+str(m_merger_error/count)+","+str(split_error)+","+str(m_split_error/count)+","+str(VI)+","+str(mVI/count)+"\n"
            out.write(line)
            out.close()

    print("average RI : ", mRI/count, "average adRI : ", mad_RI/count, "average VI : ", mVI/count, "average merger_error : ", m_merger_error/count, "average split_error : ", m_split_error/count)

# print("*"*50, " unet before ", "*"*50)
# eval_RI_VI("./iron_segment/result_total/best_unet_result/", "./evaluation/mini_RI_VI/unet_result_evaluation_mini.txt")
# print("*"*50, " unet final ", "*"*50)
# eval_RI_VI("./iron_experiments/result_total/best_unet_result_final/", "./evaluation/mini_RI_VI/unet_result_final_evaluation_mini.txt")
# print("*"*50, " unet focal ", "*"*50)
# eval_RI_VI("./iron_experiments/result_total/best_unet_result_focalloss_1/", "./evaluation/mini_RI_VI/unet_result_focalloss_1_evaluation_mini.txt")
# print("*"*50, " 3d unet ", "*"*50)
# eval_RI_VI("./3D-UNet/result_total/experiment_unet3d_01/", "./evaluation/mini_RI_VI/experiment_unet3d_01_evaluation_mini.txt")
# print("*"*50, " 3d rsunet sunperhuman ", "*"*50)
# eval_RI_VI("./3D-UNet/result_total/experiment_rsunet3d_01/", "./evaluation/mini_RI_VI/experiment_rsunet3d_01_evaluation_mini.txt")
# print("*"*50, " unet dilate reloss ", "*"*50)
# eval_RI_VI("./iron_segment_stage3/result_total/best_track_last_unet_dilate_reloss/", "./evaluation/mini_RI_VI/best_track_last_unet_dilate_reloss_evaluation_mini.txt")
print("*"*50, " unet bdclstm ", "*"*50)
eval_RI_VI("./Unet+BDCLSTM/data/test/unet_bdclstm_result/", "./evaluation/mini_RI_VI/unet_bdclstm_result_evaluation_mini.txt")


**************************************************  unet bdclstm  **************************************************
(600, 600)  gt  108  result  100
002.png  --> RI :  0.9947747723064663 adjustRI :  0.860919971651113  VI :  0.3053495588236814  merger_eroor :  0.26108271038097247  split_error :  0.04426684844270892  costs  0.04610800743103027
(600, 600)  gt  105  result  103
003.png  --> RI :  0.996012968739728 adjustRI :  0.8920641374460522  VI :  0.24618840177588247  merger_eroor :  0.18228832913925896  split_error :  0.06390007263662349  costs  0.05281233787536621
(600, 600)  gt  105  result  102
004.png  --> RI :  0.9944296406687549 adjustRI :  0.8563556389123909  VI :  0.31329125316608275  merger_eroor :  0.24870174748197468  split_error :  0.0645895056841081  costs  0.05779695510864258
(600, 600)  gt  107  result  104
005.png  --> RI :  0.9928883896282428 adjustRI :  0.8239359280845263  VI :  0.3402286239135998  merger_eroor :  0.27578425989791083  split_error :  0.06444436401568

(600, 600)  gt  104  result  112
039.png  --> RI :  0.9950619619468078 adjustRI :  0.8619935897339647  VI :  0.2664746805605998  merger_eroor :  0.11725166980822344  split_error :  0.14922301075237637  costs  0.04908633232116699
(600, 600)  gt  100  result  113
040.png  --> RI :  0.9947067680280594 adjustRI :  0.8508906822892682  VI :  0.29611705559713414  merger_eroor :  0.10635873341637013  split_error :  0.18975832218076402  costs  0.049291372299194336
(600, 600)  gt  100  result  110
041.png  --> RI :  0.9940482854890647 adjustRI :  0.8383781593398189  VI :  0.34806232918282765  merger_eroor :  0.1686468054012116  split_error :  0.17941552378161607  costs  0.049005746841430664
(600, 600)  gt  98  result  115
042.png  --> RI :  0.9949188210553671 adjustRI :  0.8615642001782482  VI :  0.2997216133014546  merger_eroor :  0.1203458932527393  split_error :  0.1793757200487153  costs  0.04938697814941406
(600, 600)  gt  96  result  104
043.png  --> RI :  0.9925956530589128 adjustRI :  0.

(600, 600)  gt  113  result  109
075.png  --> RI :  0.9970122270092726 adjustRI :  0.9175796033276472  VI :  0.186750788176476  merger_eroor :  0.12335247908390613  split_error :  0.06339830909256987  costs  0.04864311218261719
(600, 600)  gt  110  result  110
076.png  --> RI :  0.9949444277160029 adjustRI :  0.868413519001404  VI :  0.26631477311626683  merger_eroor :  0.190326918141119  split_error :  0.07598785497514782  costs  0.049094438552856445
(600, 600)  gt  123  result  116
077.png  --> RI :  0.99724919979716 adjustRI :  0.9251675988698719  VI :  0.13775970930131048  merger_eroor :  0.09163839785850778  split_error :  0.04612131144280271  costs  0.048642873764038086
(600, 600)  gt  113  result  117
078.png  --> RI :  0.9976194701775406 adjustRI :  0.934558655696901  VI :  0.12557846244688878  merger_eroor :  0.05969231043017936  split_error :  0.06588615201670942  costs  0.048651933670043945
(600, 600)  gt  111  result  120
079.png  --> RI :  0.9977341080268433 adjustRI :  0.

# F值 mIOU mAP

In [8]:
# Figure of Merit  F值
import os
import numpy as np
import cv2
from skimage import morphology

def evaluateMeritForEdge(segmentEdge, groundTruthEdge, constIndex = 0.1):
    """
    功能：使用品质因素评价方法边缘检测质量

    """
    numSeg = np.sum(segmentEdge[segmentEdge == 255])/255
    numGround = np.sum(groundTruthEdge[groundTruthEdge == 255])/255
    # print("numSeg="+str(numSeg))
    # print("numGround="+str(numGround))
    maxNum = numSeg

    if numSeg < numGround:  maxNum = numGround
    # print("maxNum="+str(maxNum))
    temp = 0.0
    for indexX in range(0, segmentEdge.shape[0]):
        for indexY in range(0, segmentEdge.shape[1]):
             if segmentEdge[indexX, indexY] == 255:
                distance = getDistanceFromGroundTruthPoint(groundTruthEdge, indexX, indexY)
                # print(str(indexX)+", "+str(indexY)+":distance="+str(distance))
                temp = temp + 1 / (1 + constIndex * pow(distance,2))
    merit = (1.0 / maxNum) * temp
    return merit

def getDistanceFromGroundTruthPoint(groundTruthEdge, indexX, indexY, neighborLength = 60):
    """
    功能：计算检测到的边缘点与离它最近边缘点的距离
    """
    if groundTruthEdge[indexX,indexY] == 255:
        return 0


    distance = neighborLength/2
    findPoint = False
    regionStartRow = 0
    regionStartCol = 0
    regionEndRow = groundTruthEdge.shape[0]
    regionEndCol = groundTruthEdge.shape[1]
    if indexX - neighborLength > 0:
        regionStartRow = indexX - neighborLength
    if indexX + neighborLength < groundTruthEdge.shape[0]:
        regionEndRow = indexX + neighborLength
    if indexY - neighborLength > 0:
        regionStartCol = indexY - neighborLength
    if indexY + neighborLength < groundTruthEdge.shape[1]:
        regionEndCol = indexY + neighborLength

    indexXX = 0;indexYY=0;
    for indexGroundX in range(regionStartRow, regionEndRow):
        for indexGroundY in range(regionStartCol, regionEndCol):
            if groundTruthEdge[indexGroundX, indexGroundY] == 255:
                tempDisance = np.sqrt(pow(indexGroundX - indexX,2)+pow(indexGroundY - indexY,2))
                if tempDisance < distance:
                    distance = tempDisance
                    indexXX = indexGroundX
                    indexYY = indexGroundY
    # print("     indexXX="+str(indexXX)+",indexYY="+str(indexYY))
    return distance

import cv2
import numpy as np
import skimage.measure

def evaluateAP(pred, gt, neighbors = 4, bgPixel = 0, iouThreshold = 0.5):
    '''
    通过IOU计算AP，参考
    https://mp.weixin.qq.com/s?__biz=MzU1NTUxNTM0Mg==&mid=2247488695&idx=2&sn=05b55ac4a45cd1549f2fa201271c3b54&chksm=fbd27816cca5f1009c677b7c12f377672673826d4facc6f551e2158b6d4cf0fc5beaffb043c9&mpshare=1&scene=1&srcid=0522GkUltOSpZJKQT2ZAxnqz#rd
    :param pred:预测图
    :param gt: 真值图
    :param neighbors:4临接或8临接
    :param bgPixel:背景像素，针对铝镧枝晶为0，针对纯铁晶粒为255或1
    :param iouThreshold:IOU的阈值，大于此阈值truePositive自加1
    :return:返回AP
    '''
    # 标记预测图与真值图
    predLabel, predNum = skimage.measure.label(pred, neighbors=neighbors, background=bgPixel, return_num=True)
    gtLabel, gtNum = skimage.measure.label(gt, neighbors=neighbors, background=bgPixel, return_num=True)

    truePositives = 0       # 记录Iou > Threshold 的数目
    iou = 0
    for i in range(1, predNum + 1):
        interLabel = list(np.unique(gtLabel[predLabel == i]))   # 获得与之相交的所有label
        interLabelArea = []
        if 0 in interLabel:
            interLabel.remove(0)

        if len(interLabel) == 0:        # 如果pred的某一个label没有与之对应的gt的label,则继续下一个label
            continue

        for j in range(len(interLabel)):
            interLabelArea.append(np.count_nonzero(predLabel[gtLabel == interLabel[j]] == i))

        intersectLabel = interLabel[interLabelArea.index(max(interLabelArea))]  # 获得与之相交面积最大的gtlabel

        # 获得与之相交面积最大的gtlabel面积,iou的分子
        intersection = max(interLabelArea)

        # 计算Union，iou的分母
        union = 0
        for j in range(predLabel.shape[0]):
            for k in range(predLabel.shape[1]):
                if gtLabel[j, k] == intersectLabel or predLabel[j, k] == i:
                    union = union + 1

        # 如果IOU大于一定阈值，truePositives自加1
        iou += intersection/union
        if intersection/union > iouThreshold:
            truePositives = truePositives + 1
    return iou/predNum, truePositives/gtNum

In [ ]:
# 计算88张测试集的品质因素
# "./iron_segment/result_total/best_unet_result/" -> unet_result_evaluation_mini
# "./iron_experiments/result_total/best_unet_result_final/" -> unet_result_final_evaluation_mini

# "./iron_experiments/result_total/best_unet_result_focalloss_1/" -> unet_result_focalloss_1_evaluation_mini
# "./3D-UNet/result_total/experiment_unet3d_01/" -> experiment_unet3d_01_evaluation_mini   3d unet
# "./3D-UNet/result_total/experiment_rsunet3d_01/"  -> experiment_rsunet3d_01_evaluation_mini   3d  rs_unet  superhuman
# # "./iron_segment_stage3/result_total/best_track_last_unet_dilate_reloss"  ->  best_track_last_unet_dilate_reloss_evaluation_mini


import time

results_dir = "./3D-UNet/result_total/experiment_unet3d_01/"
results_path =  sorted(os.listdir(results_dir))
print("images count: ", len(results_path))

gt_dir = "./Unet+BDCLSTM/data/test/manual/"
F = 0
F_test = 0
mIOU = 0
mAP = 0
count = 0
outAd = "./evaluation/mini_F_mIOU_mAP/experiment_unet3d_01_evaluation_mini.txt"
out = open(outAd, "a")   # 将结果输出至out_lg.txt
out.write("F,avF,mIOU,avmIOU,mAP,avmAP" + '\n')
out.close()
for r in results_path:
    name = r.split(".")[0]
    gt_path = os.path.join(gt_dir, name+".png")
    if os.path.exists(gt_path):
        print("image", name)
        count += 1
        result = cv2.imread(os.path.join(results_dir, r), 0)
        gt = cv2.imread(gt_path, 0)
        
        shape = result.shape
        
        result = result[400:1000, 500:1100]
        gt = gt[400:1000, 500:1100]
        
        print(result.shape)
        

#         result = morphology.skeletonize(result / 255) * 255

        start_time = time.time()
        F_test = evaluateMeritForEdge(result, gt)
        end_time = time.time()
        F += F_test
        print(count, " ", r, " --> F : ", F_test, " avF : ", F/count, " costs ", str(end_time-start_time))

        start_time = time.time()
        mIOU_test, mAP_test = evaluateAP(result, gt, neighbors=4, bgPixel=255)
        end_time = time.time()
        mIOU += mIOU_test
        mAP += mAP_test
        print(count, " ", r, " --> mIOU : ", mIOU_test, " avmIOU : ", mIOU/count, " mAP : ", mAP_test, " avmAP : ", mAP/count, " costs ", str(end_time-start_time))

        out = open(outAd, "a")   # 将结果输出至out_lg.txt
        line = str(F_test)+","+str(F/count)+","+str(mIOU_test)+","+str(mIOU/count)+","+str(mAP_test)+","+str(mAP/count)+"\n"
        out.write(line)
        out.close()
    
F = F/count
mIOU = mIOU/count
mAP = mAP/count
print("average F : ", F, "average mIOU : ", mIOU, "average mAP : ", mAP)
    

images count:  88
image 001
(600, 600)
1   001.png  --> F :  0.9158371125537581  avF :  0.9158371125537581  costs  131.17847847938538
1   001.png  --> mIOU :  0.7783888598062488  avmIOU :  0.7783888598062488  mAP :  0.8545454545454545  avmAP :  0.8545454545454545  costs  22.31701922416687
image 002
(600, 600)
2   002.png  --> F :  0.9111458281774558  avF :  0.913491470365607  costs  129.30285787582397
2   002.png  --> mIOU :  0.828764833201901  avmIOU :  0.8035768465040749  mAP :  0.8981481481481481  avmAP :  0.8763468013468013  costs  22.141392469406128
image 003
(600, 600)
4   004.png  --> F :  0.9103315355663704  avF :  0.8806852028527161  costs  125.78794503211975
4   004.png  --> mIOU :  0.7980727063748307  avmIOU :  0.7201597011770144  mAP :  0.8666666666666667  avmAP :  0.8810305435305436  costs  21.113199710845947
image 005
(600, 600)
5   005.png  --> F :  0.8826055083720122  avF :  0.8810692639565753  costs  134.41454768180847
5   005.png  --> mIOU :  0.7625491836559322  avmIO

# mAP New

In [3]:
import cv2
import numpy as np
import skimage.measure
import os
from skimage import morphology

def evaluateAP_new(pred, gt, neighbors = 4, bgPixel = 0, iouThreshold = 0.5):
    '''
    通过IOU计算AP，参考
    https://mp.weixin.qq.com/s?__biz=MzU1NTUxNTM0Mg==&mid=2247488695&idx=2&sn=05b55ac4a45cd1549f2fa201271c3b54&chksm=fbd27816cca5f1009c677b7c12f377672673826d4facc6f551e2158b6d4cf0fc5beaffb043c9&mpshare=1&scene=1&srcid=0522GkUltOSpZJKQT2ZAxnqz#rd
    :param pred:预测图
    :param gt: 真值图
    :param neighbors:4临接或8临接
    :param bgPixel:背景像素，针对铝镧枝晶为0，针对纯铁晶粒为255或1
    :param iouThreshold:IOU的阈值，大于此阈值truePositive自加1
    :return:返回AP
    '''
    # 标记预测图与真值图
    predLabel, predNum = skimage.measure.label(pred, neighbors=neighbors, background=bgPixel, return_num=True)
    gtLabel, gtNum = skimage.measure.label(gt, neighbors=neighbors, background=bgPixel, return_num=True)

    truePositives = 0       # 记录Iou > Threshold 的数目
    mAP = 0  # 记录加权mAP
    iou = 0
    for i in range(1, predNum + 1):
        interLabel = list(np.unique(gtLabel[predLabel == i]))   # 获得与之相交的所有label
        interLabelArea = []
        if 0 in interLabel:
            interLabel.remove(0)

        if len(interLabel) == 0:        # 如果pred的某一个label没有与之对应的gt的label,则继续下一个label
            continue

        for j in range(len(interLabel)):
            interLabelArea.append(np.count_nonzero(predLabel[gtLabel == interLabel[j]] == i))

        intersectLabel = interLabel[interLabelArea.index(max(interLabelArea))]  # 获得与之相交面积最大的gtlabel
        
        weight = np.count_nonzero(gtLabel == intersectLabel)/360000  # 求对应面积的权重

        # 获得与之相交面积最大的gtlabel面积,iou的分子
        intersection = max(interLabelArea)

        # 计算Union，iou的分母
        union = 0
        for j in range(predLabel.shape[0]):
            for k in range(predLabel.shape[1]):
                if gtLabel[j, k] == intersectLabel or predLabel[j, k] == i:
                    union = union + 1

        # 如果IOU大于一定阈值，truePositives自加1
        iou += intersection/union
        if intersection/union > iouThreshold:
            truePositives = truePositives + 1
            mAP = mAP + weight
    return truePositives/gtNum, mAP

In [4]:
%%time
# 计算88张测试集的品质因素


import time

results_dir = "./3D-UNet/result_total/experiment_unet3d_01/"
results_path =  sorted(os.listdir(results_dir))
print("images count: ", len(results_path))

gt_dir = "./Unet+BDCLSTM/data/test/manual/"
mAP_test = 0
mAP_new_test = 0
mAP = 0
mAP_new = 0 
count = 0

for r in results_path:
    name = r.split(".")[0]
    gt_path = os.path.join(gt_dir, name+".png")
    if os.path.exists(gt_path):
        print("image", name)
        count += 1
        result = cv2.imread(os.path.join(results_dir, r), 0)
        gt = cv2.imread(gt_path, 0)
        
        shape = result.shape
        
        result = result[400:1000, 500:1100]
        gt = gt[400:1000, 500:1100]
        
        print(result.shape)
        

        result = morphology.skeletonize(result / 255) * 255

        start_time = time.time()
        mAP_test, mAP_new_test = evaluateAP_new(result, gt, neighbors=4, bgPixel=255, iouThreshold=0.7)
        end_time = time.time()
        mAP += mAP_test
        mAP_new += mAP_new_test
        print(count, " ", r, " --> mAP : ", mAP_test, " avmAP : ", mAP/count, " mAP_new : ", mAP_new_test, " avmAP_new : ", mAP_new/count, " costs ", str(end_time-start_time))
    
mAP = mAP/count
mAP_new = mAP_new/count
print("average mAP7 : ", mAP, "average mAP7_new : ", mAP_new)
    

images count:  88
image 001
(600, 600)
1   001.png  --> mAP :  0.8727272727272727  avmAP :  0.8727272727272727  mAP_new :  0.9355666666666665  avmAP_new :  0.9355666666666665  costs  23.388485193252563
image 002
(600, 600)
2   002.png  --> mAP :  0.9259259259259259  avmAP :  0.8993265993265993  mAP_new :  0.93195  avmAP_new :  0.9337583333333332  costs  22.57399559020996
image 003
(600, 600)
3   003.png  --> mAP :  0.8285714285714286  avmAP :  0.8757415424082091  mAP_new :  0.8649611111111108  avmAP_new :  0.9108259259259258  costs  21.619744300842285
image 004
(600, 600)
4   004.png  --> mAP :  0.8761904761904762  avmAP :  0.8758537758537759  mAP_new :  0.9060555555555556  avmAP_new :  0.9096333333333333  costs  22.090860843658447
image 005
(600, 600)
5   005.png  --> mAP :  0.8317757009345794  avmAP :  0.8670381608699366  mAP_new :  0.9001027777777777  avmAP_new :  0.9077272222222221  costs  23.850204706192017
image 006
(600, 600)
6   006.png  --> mAP :  0.8504672897196262  avmAP :  

47   047.png  --> mAP :  0.8761904761904762  avmAP :  0.8666834244681558  mAP_new :  0.8249083333333332  avmAP_new :  0.8841013593380612  costs  21.223661184310913
image 048
(600, 600)
48   048.png  --> mAP :  0.7456140350877193  avmAP :  0.86416114552273  mAP_new :  0.6951972222222222  avmAP_new :  0.8801658564814812  costs  22.89241123199463
image 049
(600, 600)
49   049.png  --> mAP :  0.7235772357723578  avmAP :  0.8612920861400695  mAP_new :  0.7158833333333334  avmAP_new :  0.8768131519274373  costs  22.727472066879272
image 050
(600, 600)
50   050.png  --> mAP :  0.808695652173913  avmAP :  0.8602401574607462  mAP_new :  0.7365583333333331  avmAP_new :  0.8740080555555552  costs  23.39050793647766
image 051
(600, 600)
51   051.png  --> mAP :  0.8620689655172413  avmAP :  0.860276016442246  mAP_new :  0.8439777777777779  avmAP_new :  0.8734192265795204  costs  22.424978256225586
image 052
(600, 600)
52   052.png  --> mAP :  0.47368421052631576  avmAP :  0.8528415586361705  mAP_ne